In [9]:
import pandas as pd
import plotly.graph_objects as go

# Load and prepare the data
df = pd.read_csv(r'amiNF_trd_sig_backtest_test.csv')
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['TradeEntryTime'])
df.sort_values(by=['Datetime'], inplace=True)
df.reset_index(drop=True, inplace=True)
df['Trade_No'] = df.index + 1

margin_per_lot = 80000  # 80k margin per lot
lot_size = 50

# Starting Capital
start_capital = 1800000  # 18 lacs

# Initialize empty lists for the calculated fields
Margin_utilized = []
Lots = []
Qty = []
PnL = []
Taxes = []
NetPnL = []
Capital = [start_capital]

# Calculate additional columns
for i in range(len(df)):
    Margin_utilized.append(0.6 * Capital[-1])
    Lots.append(Margin_utilized[-1] // margin_per_lot)
    Qty.append(int(Lots[-1] * lot_size))
    PnL.append(Qty[-1] * df.loc[i, 'NetTradePoints'])
    Taxes.append(0.1 * abs(PnL[-1]))
    NetPnL.append(PnL[-1] - Taxes[-1])
    if i < len(df) - 1:  # Avoids IndexError on the last row
        Capital.append(Capital[-1] + NetPnL[-1])

# Update the DataFrame with the calculated fields
df['Margin_utilized'] = Margin_utilized
df['Lots'] = Lots
df['Qty'] = Qty
df['PnL'] = PnL
df['Taxes'] = Taxes
df['NetPnL'] = NetPnL
df['Capital'] = Capital
df.drop(columns=['Datetime'], inplace=True)

# Create plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['NetPnL'].cumsum(), mode='lines+markers', name='NetPnL'))
fig.update_layout(title='Net Profit and Loss Over Time', xaxis_title='Date', yaxis_title='Net PnL',
                  autosize=False, width=1000, height=500)
fig.show()


In [12]:
df.to_csv(r'amiNF_mini_sim.csv', index=False)

In [14]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from scipy.stats import norm

# Load and prepare the data
df = pd.read_csv(r'amiNF_trd_sig_backtest_test.csv')
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['TradeEntryTime'])
df.sort_values(by=['Datetime'], inplace=True)
df.reset_index(drop=True, inplace=True)
df['Trade_No'] = df.index + 1

margin_per_lot = 80000  # 80k margin per lot
lot_size = 50

# Starting Capital
start_capital = 1800000  # 18 lacs

# Initialize empty lists for the calculated fields
Margin_utilized = []
Lots = []
Qty = []
PnL = []
Taxes = []
NetPnL = []
Capital = [start_capital]

# Calculate additional columns
for i in range(len(df)):
    Margin_utilized.append(0.6 * Capital[-1])
    Lots.append(Margin_utilized[-1] // margin_per_lot)
    Qty.append(int(Lots[-1] * lot_size))
    PnL.append(Qty[-1] * df.loc[i, 'NetTradePoints'])
    Taxes.append(0.1 * abs(PnL[-1]))
    NetPnL.append(PnL[-1] - Taxes[-1])
    if i < len(df) - 1:  # Avoids IndexError on the last row
        Capital.append(Capital[-1] + NetPnL[-1])

# Update the DataFrame with the calculated fields
df['Margin_utilized'] = Margin_utilized
df['Lots'] = Lots
df['Qty'] = Qty
df['PnL'] = PnL
df['Taxes'] = Taxes
df['NetPnL'] = NetPnL
df['Capital'] = Capital

# Additional statistics
annual_return = (df['NetPnL'].sum() / start_capital) * 100  # in percentage
net_profit_percentage = ((Capital[-1] / start_capital) - 1) * 100  # in percentage
avg_profit_loss = df['PnL'].mean()
largest_win = df['PnL'].max()
largest_loss = df['PnL'].min()
largest_win_trade_no = df['PnL'].idxmax() + 1  # trade numbers start at 1
largest_loss_trade_no = df['PnL'].idxmin() + 1
max_consecutive_wins = (df['PnL'] > 0).astype(int).groupby(df['PnL'].astype(bool).cumsum()).count().max()
max_consecutive_losses = (df['PnL'] < 0).astype(int).groupby(df['PnL'].astype(bool).cumsum()).count().max()
# Calculate drawdowns and max drawdown
drawdowns = pd.Series(Capital) - pd.Series(Capital).cummax()
max_drawdown = -drawdowns.min()
max_percent_drawdown = (max_drawdown / pd.Series(Capital).loc[drawdowns.idxmin()]) * 100  # in percentage

recovery_factor = df['NetPnL'].sum() / max_drawdown
risk_reward_ratio = abs(df['PnL'][df['PnL'] > 0].mean() / df['PnL'][df['PnL'] < 0].mean())
sharpe_ratio = df['NetPnL'].mean() / df['NetPnL'].std() * np.sqrt(len(df))  # assume risk-free rate = 0

# Print statistics
print(f"Annual Return (%): {annual_return}")
print(f"Net Profit (%): {net_profit_percentage}")
print(f"Average Profit/Loss: {avg_profit_loss}")
print(f"Largest Win: {largest_win}")
print(f"Largest Win Trade No: {largest_win_trade_no}")
print(f"Largest Loss: {largest_loss}")
print(f"Largest Loss Trade No: {largest_loss_trade_no}")
print(f"Max Consecutive Wins: {max_consecutive_wins}")
print(f"Max Consecutive Losses: {max_consecutive_losses}")
print(f"Max Drawdown: {max_drawdown}")
print(f"Max % Drawdown: {max_percent_drawdown}")
print(f"Recovery Factor: {recovery_factor}")
print(f"Risk/Reward Ratio: {risk_reward_ratio}")
print(f"Sharpe Ratio: {sharpe_ratio}")

df.drop(columns=['Datetime'], inplace=True)

# Create plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['NetPnL'].cumsum(), mode='lines+markers', name='NetPnL'))
fig.update_layout(title='Net Profit and Loss Over Time', xaxis_title='Date', yaxis_title='Net PnL',
                  autosize=False, width=1000, height=500)
fig.show()


Annual Return (%): 2.202777777777778
Net Profit (%): 1.838777777777767
Average Profit/Loss: 8970.000000000002
Largest Win: 34157.5
Largest Win Trade No: 4
Largest Loss: -3575.0
Largest Loss Trade No: 3
Max Consecutive Wins: 1
Max Consecutive Losses: 1
Max Drawdown: 3932.5
Max % Drawdown: 0.21818660988913818
Recovery Factor: 10.082644628099175
Risk/Reward Ratio: 3.3863636363636367
Sharpe Ratio: 1.314198642365173


In [ ]:
df.to_csv(r'amiNF_mini_sim.csv', index=False)